##### EEEL 4220 Final Project
## Wind Farm Sizing and Siting on IEEE-14 System

1. Single-bus single-scenario, no wind planning. Start assuming all buses one the same
bus, minimize generation cost from a single scenario with one power balance constraint and
the limit of each generator.

In [302]:
# Import libraries
import cvxpy as cp
import numpy as np
import pandas as pd

In [303]:
# Load data
g_df = pd.read_csv('generators.csv')
d_df = pd.read_csv('demand_scenarios.csv')

In [304]:
# Parameters
c0 = g_df['c0'].to_numpy() # quadratic cost coefficients for generator g
c1 = g_df['c1'].to_numpy() # linear cost coefficients for generator g
c2 = g_df['c2'].to_numpy() # fixed cost coefficients for generator g
Pmin = g_df['Pmin_MW'].to_numpy() # minimum generation limits for generator g
Pmax = g_df['Pmax_MW'].to_numpy() # maximum generation limits for generator g
D = d_df.loc[d_df['scenario_id']==1,'Pd_MW'].sum() # demand at bus b in scenario s

# Decision variables
p = cp.Variable(len(g_df), nonneg=True)

In [305]:
# Define constraints
# Initialize an empty constraint set
con = [] 

# power balance
con.append(D == cp.sum(p))

# generator output limits
con.append(p >= Pmin)
con.append(p <= Pmax)

In [306]:
# Define objective function - total cost
obj = cp.Minimize(cp.sum(cp.multiply(c2, cp.square(p)) + cp.multiply(c1, p) + c0))

In [307]:
# Solve the problem
prob1 = cp.Problem(obj, con)
prob1.solve(solver = "Gurobi");

In [309]:
# results
print("\n The total generation in the system is:")
print(p.value.sum().round(1))
print("The total demand in the system is:") 
print(D.sum().round(1))

print("\n Expected operating cost:")
print(obj.value.round(1))

print("\n Thermal generator dispatch results: ")
GT = pd.DataFrame(p.value, index = ['G1', 'G2', 'G3', 'G4', 'G5'])
print(GT.round(1))


 The total generation in the system is:
233.1
The total demand in the system is:
233.1

 Expected operating cost:
721.2

 Thermal generator dispatch results: 
       0
G1  79.1
G2  42.9
G3  43.5
G4  35.9
G5  31.7
